## Deliverable 2. Create a Customer Travel Destinations Map.

In [11]:
# Dependencies and Setup
import pandas as pd
import numpy as np
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Kodiak,US,57.7900,-152.4072,38.71,75,100,6.91,light snow
1,1,Cape Town,ZA,-33.9258,18.4232,60.28,69,0,20.71,clear sky
2,2,Asau,RO,46.4333,26.4000,45.95,82,100,4.74,overcast clouds
3,3,Katobu,ID,-4.9408,122.5276,74.79,93,85,1.54,overcast clouds
4,4,Bluff,NZ,-46.6000,168.3333,56.52,65,91,27.38,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimun temperature you would like for your trip? "))
max_temp = float(input("What is the maximun temperature you would like for your trip? "))

What is the minimun temperature you would like for your trip? 65
What is the maximun temperature you would like for your trip? 95


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
cities_preferred_df = city_data_df.loc[(city_data_df["Max Temp"] >= min_temp) & (city_data_df["Max Temp"] <= max_temp)]
cities_preferred_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
3,3,Katobu,ID,-4.9408,122.5276,74.79,93,85,1.54,overcast clouds
7,7,Sitio Novo Do Tocantins,BR,-5.5183,-47.4611,76.78,88,15,2.28,few clouds
9,9,Kalmunai,LK,7.4167,81.8167,78.89,82,83,3.87,broken clouds
10,10,Adrar,MR,20.5022,-10.0711,75.60,16,0,11.34,clear sky
12,12,Boyolangu,ID,-4.9408,122.5276,74.79,93,85,1.54,overcast clouds


In [7]:
# 4a. Determine if there are any empty rows.
cities_preferred_df.isnull().sum()
cities_preferred_df.count()

City_ID                215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = cities_preferred_df.dropna(axis="index", how="any")
clean_df.head()
clean_df.count()

City_ID                215
City                   215
Country                215
Lat                    215
Lng                    215
Max Temp               215
Humidity               215
Cloudiness             215
Wind Speed             215
Current Description    215
dtype: int64

In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Katobu,ID,74.79,overcast clouds,-4.9408,122.5276,
7,Sitio Novo Do Tocantins,BR,76.78,few clouds,-5.5183,-47.4611,
9,Kalmunai,LK,78.89,broken clouds,7.4167,81.8167,
10,Adrar,MR,75.60,clear sky,20.5022,-10.0711,
12,Boyolangu,ID,74.79,overcast clouds,-4.9408,122.5276,
14,Axim,GH,81.18,overcast clouds,4.8699,-2.2405,
15,Ballina,AU,68.02,overcast clouds,-28.8667,153.5667,
20,San Policarpo,PH,77.47,moderate rain,12.1791,125.5072,
22,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,
35,Fonte Boa,BR,76.78,few clouds,-5.5183,-47.4611,


In [12]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping the city.")
        hotel_df.loc[index, "Hotel Name"] = np.nan

Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.
Hotel not found...skipping the city.


In [13]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis="index", how="any")
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
3,Katobu,ID,74.79,overcast clouds,-4.9408,122.5276,La Ode Husein
7,Sitio Novo Do Tocantins,BR,76.78,few clouds,-5.5183,-47.4611,Imperial Hotel
9,Kalmunai,LK,78.89,broken clouds,7.4167,81.8167,VS Villa
12,Boyolangu,ID,74.79,overcast clouds,-4.9408,122.5276,La Ode Husein
14,Axim,GH,81.18,overcast clouds,4.8699,-2.2405,Axim Beach Resort And Spa
15,Ballina,AU,68.02,overcast clouds,-28.8667,153.5667,Ramada Hotel & Suites by Wyndham Ballina Byron
20,San Policarpo,PH,77.47,moderate rain,12.1791,125.5072,Agapita M. Nicart
22,Vaini,TO,82.56,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
35,Fonte Boa,BR,76.78,few clouds,-5.5183,-47.4611,Imperial Hotel
40,Kavaratti,IN,83.07,broken clouds,10.5669,72.6420,Hotels in Lakshadweep Islands


In [14]:
# 8a. Create the output File (CSV)
output_data_file = "Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [15]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""


# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [16]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))